In [6]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [50]:
df = pd.read_csv("data/Form.csv")


# Prepare data

In [51]:
# remove data not use
df = df.iloc[0:,[1,2,3]]

In [52]:
# we use age , gender and trasactions Store
df.head(5)

,Age,Gender,Store
0,20,male,"NAP's Coffee & Roasters, sangob, Nap x Warin, ..."
1,16,female,"BalconyKiss Coffee, ROOF COFFEE, LAVA JAVA Co..."
2,22,male,"NAP's Coffee & Roasters, SongSarn, Yuanjai Cafe"
3,21,female,"NAP's Coffee & Roasters, Nap x Warin, Snoopcat..."
4,19,male,"NAP's Coffee & Roasters, sangob, Nap x Warin, ..."


In [53]:
# create functions to split transactions data store

def set_transactions(x):
    x = x.split(",")
    return x

In [54]:
# map values with age gender and store to format functions asscoations rule
df["Store"]  = df["Store"].apply(lambda x:set_transactions(x))


In [55]:
# map Age 
df['Age'] = df["Age"].map(lambda x:'10-20' if x <= 20 else '21-40' if x <= 40 else '41-60' if x <= 60 else '60+')

In [56]:
# map age gender to create Rule
age_groups = df.groupby('Age')
age_gender_rules = {}

In [57]:
# For Loop each group
for age_group, data in age_groups:
    for gender, gender_data in data.groupby('Gender'):
        # Use the TransactionEncoder to convert the data into a suitable format for the Apriori algorithm
        te = TransactionEncoder()
        te_ary = te.fit(gender_data['Store']).transform(gender_data['Store'])
        df_temp = pd.DataFrame(te_ary, columns=te.columns_)

        # Run the Apriori algorithm to find frequent item sets
        frequent_itemsets = apriori(df_temp, min_support=0.05, use_colnames=True)

        # Generate association rules from the frequent item sets
        rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
        # Store the association rules for each age group and gender in the dictionary
        age_gender_rules[(age_group, gender)] = rules

In [58]:
age_gender_rules.keys()

dict_keys([('10-20', 'female'), ('10-20', 'male'), ('21-40', 'female'), ('21-40', 'male'), ('41-60', 'female'), ('41-60', 'male')])